# 03 - About $\textsf{RLWE}'$
---

From <a href='https://eprint.iacr.org/2020/086'> Bootstrapping in FHEW-like Cryptosystems </a>

In [1]:
import numpy as np
from note_include.elem.Ring  import Ring
from note_include.elem.RLWE  import RLWE
from note_include.elem.RLWEp import RLWEp
from note_include.utils.gadget_decomposition import gadget_decomposition, gadget_composition, format_ring_list

---
#### Parameters


In [2]:
n   = 16
q   = 7681
B   = 16
d   = int(np.ceil(np.log(q) / np.log(B)))
std = 3.2

---
Test Gadget (De)Composition

In [3]:
coeffs = [i * 100 for i in range(n)]
poly   = Ring(n, q, coeffs)
decomposed_polys = gadget_decomposition(poly, B, int(np.ceil(np.log(q) / np.log(B))) )
composed_poly    = gadget_composition(decomposed_polys, B, q)

print('Plain value : ', coeffs)
print('---- Decomposed polynomials ----')
print(format_ring_list(decomposed_polys))
print('---- Composed polynomials ----')
print(composed_poly)

Plain value :  [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500]
---- Decomposed polynomials ----
R(n=16, q=7681, coeffs= 0 + 4x + 8x^2 + 12x^3 + 0x^4 + 4x^5 + 8x^6 + 12x^7 + 0x^8 + 4x^9 + 8x^10 + 12x^11 + 0x^12 + 4x^13 + 8x^14 + 12x^15)
R(n=16, q=7681, coeffs= 0 + 6x + 12x^2 + 2x^3 + 9x^4 + 15x^5 + 5x^6 + 11x^7 + 2x^8 + 8x^9 + 14x^10 + 4x^11 + 11x^12 + 1x^13 + 7x^14 + 13x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 1x^3 + 1x^4 + 1x^5 + 2x^6 + 2x^7 + 3x^8 + 3x^9 + 3x^10 + 4x^11 + 4x^12 + 5x^13 + 5x^14 + 5x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 0x^12 + 0x^13 + 0x^14 + 0x^15)
---- Composed polynomials ----
R(n=16, q=7681, coeffs= 0 + 100x + 200x^2 + 300x^3 + 400x^4 + 500x^5 + 600x^6 + 700x^7 + 800x^8 + 900x^9 + 1000x^10 + 1100x^11 + 1200x^12 + 1300x^13 + 1400x^14 + 1500x^15)


---
Encryption and Multiplication with Constant

In [4]:
def trim_zeros(lst):
    """ 리스트의 앞뒤에서 연속된 0을 제거 """
    while lst and lst[0] == 0:  # 앞쪽 0 제거
        lst.pop(0)
    while lst and lst[-1] == 0:  # 뒤쪽 0 제거
        lst.pop()
    return

In [5]:
for i in range(10):
    B = 4
    d   = int(np.ceil(np.log(q) / np.log(B)))

    rlwep = RLWEp(n, q, std, B, d)
    rlwe  = RLWE(n, q, std)

    sk, pk     = rlwe.keygen()
    coeffs     = [i for i in range(n)]
    big_const  = np.random.randint(0, q)
    # big_const  = 1141
    poly       = Ring(n, q, coeffs)

    rlwep_ctxts     = rlwep.encrypt(poly, sk)
    rlwe_ctxt       = rlwe.encrypt(poly, sk)
    ctxt_rlwep_mult = rlwep.mult_constant(rlwep_ctxts, big_const)
    ctxt_rlwe_mult  = rlwe.mult_ring_cnst(rlwe_ctxt, big_const)

    ptxt_rlwep = rlwe.decrypt(ctxt_rlwep_mult, sk)
    ptxt_rlwe  = rlwe.decrypt(ctxt_rlwe_mult,  sk)
    ideal      = big_const * poly

    if trim_zeros(ptxt_rlwep.coeffs) != trim_zeros(ideal.coeffs):
        print("Ideal Result     : ", ideal)
        print("Plaintext(RLWEp) : ", ptxt_rlwep)
        break

print("Multiplier       : ", big_const)
print("Message          : ", poly)
print("Ideal Result     : ", big_const * poly)
print("Plaintext(RLWEp) : ", ptxt_rlwep)
print("Plaintext(RLWE)  : ", ptxt_rlwe)

in rlwep encrypt msg       :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
in rlwep encrypt B^i * msg :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
decryption in rlwep calss  :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
in rlwep encrypt msg       :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
in rlwep encrypt B^i * msg :  R(n=16, q=7681, coeffs= 0 + 4x + 8x^2 + 12x^3 + 16x^4 + 20x^5 + 24x^6 + 28x^7 + 32x^8 + 36x^9 + 40x^10 + 44x^11 + 48x^12 + 52x^13 + 56x^14 + 60x^15)
decryption in rlwep calss  :  R(n=16, q=7681, coeffs= 0 + 4x + 8x^2 + 12x^3 + 16x^4 + 20x^5 + 24x^6 + 28x^7 + 32x^8 + 36x^9 + 40x^10 

---
Encryption and Multiplication with polynomial

In [6]:
for i in range(10):    
    B = 4
    d   = int(np.ceil(np.log(q) / np.log(B)))
    std = 0
    
    rlwep = RLWEp(n, q, std, B, d)
    rlwe  = RLWE(n, q, std)
    
    sk, pk          = rlwe.keygen()
    coeffs          = [i for i in range(n)]
    big_coeffs      = [300 for _ in range(n)]
    big_coeffs      = [0 for _ in range(n)]
    # big_coeffs[0]   = np.random.randint(0, q)
    poly            = Ring(n, q, coeffs)
    poly_multiplier = Ring(n, q, big_coeffs)

    rlwep_ctxts     = rlwep.encrypt(poly, sk)
    rlwe_ctxt       = rlwe.encrypt(poly, sk)
    ctxt_rlwep_mult = rlwep.mult_poly(rlwep_ctxts, poly_multiplier)
    ctxt_rlwe_mult  = rlwe.mult_ring_ptxt(rlwe_ctxt, poly_multiplier)
    
    ptxt_rlwep = rlwe.decrypt(ctxt_rlwep_mult, sk)
    ptxt_rlwe  = rlwe.decrypt(ctxt_rlwe_mult,  sk)
    ideal      = poly * poly_multiplier

    if trim_zeros(ptxt_rlwep.coeffs) == trim_zeros(ideal.coeffs):
        print("Ideal Result     : ", ideal)
        print("Plaintext(RLWEp) : ", ptxt_rlwep)
        break


# print("Multiplier       : ", big_const)
# print("Message          : ", poly)
# print("Ideal Result     : ", poly * poly_multiplier)
# print("Plaintext(RLWEp) : ", ptxt_rlwep)
# print("Plaintext(RLWE)  : ", ptxt_rlwe)

in rlwep encrypt msg       :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
in rlwep encrypt B^i * msg :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
decryption in rlwep calss  :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
in rlwep encrypt msg       :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
in rlwep encrypt B^i * msg :  R(n=16, q=7681, coeffs= 0 + 4x + 8x^2 + 12x^3 + 16x^4 + 20x^5 + 24x^6 + 28x^7 + 32x^8 + 36x^9 + 40x^10 + 44x^11 + 48x^12 + 52x^13 + 56x^14 + 60x^15)
decryption in rlwep calss  :  R(n=16, q=7681, coeffs= 0 + 4x + 8x^2 + 12x^3 + 16x^4 + 20x^5 + 24x^6 + 28x^7 + 32x^8 + 36x^9 + 40x^10 

In [17]:
n   = 16
q   = 16
B   = 1024
std = 0
d   = int(np.ceil(np.log(q) / np.log(B)))

rlwep = RLWEp(n, q, std, B, d)
rlwe  = RLWE(n, q, std)

sk = [1, 3, 13, 2, 0, 14, 14, 1, 3, 14, 3, 12, 0, 4, 2, 15]
aa = [12, 7, 6, 4, 13, 5, 8, 11, 7, 6, 15, 2, 15, 5, 12, 1]
mo = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

sk = Ring(n, q, sk)
aa = Ring(n, q, aa)
mo = Ring(n, q, mo)

# akkkk = rlwep.encrypt(-1 * sk * mo, sk)

# aaa, bbb = akkkk[0]
# print("decryption of rlwep(-sm) : ",rlwe.decrypt(akkkk[0], sk).coeffs)

# res   = rlwep.mult_poly(akkkk, aa)
# res   = rlwe.decrypt(res, sk)


akkkk_rlwe = rlwe.encrypt(-1 * sk * mo, sk)
a, b = akkkk_rlwe
print(" a : ", a)
print(" b : ", b)


res2   = rlwe.mult_ring_ptxt(akkkk_rlwe, aa)
res2   = rlwe.decrypt(res2, sk)

print("Ideal (-sm)   : ", -1 * sk * mo)
print("Ideal (-asm)  : ", -1 * sk * aa * mo)
# print("Result(RLWEp) : ", res)
print("Result(RLWE)  : ", res2)

 a :  R(n=16, q=16, coeffs= 13 + 12x + 1x^2 + 14x^3 + 15x^4 + 10x^5 + 3x^6 + 11x^7 + 10x^8 + 15x^9 + 5x^10 + 8x^11 + 15x^12 + 6x^13 + 10x^14 + 8x^15)
 b :  R(n=16, q=16, coeffs= 15 + 13x + 4x^2 + 3x^3 + 1x^4 + 11x^5 + 0x^6 + 5x^7 + 12x^8 + 14x^9 + 9x^10 + 3x^11 + 12x^12 + 15x^13 + 8x^14)
-s :  R(n=16, q=16, coeffs= 4.0 + 2.0x + 15.0x^2 + 15.0x^3 + 13.0x^4 + 3.0x^5 + 14.0x^6 + 0.0x^7 + 2.0x^8 + 2.0x^9 + 15.0x^10 + 13.0x^11 + 2.0x^12 + 13.0x^13 + 4.0x^14)
Ideal (-sm)   :  R(n=16, q=16, coeffs= 4.0 + 2.0x + 15.0x^2 + 15.0x^3 + 13.0x^4 + 3.0x^5 + 14.0x^6 + 0.0x^7 + 2.0x^8 + 2.0x^9 + 15.0x^10 + 13.0x^11 + 2.0x^12 + 13.0x^13 + 4.0x^14)
Ideal (-asm)  :  R(n=16, q=16, coeffs= 1.0 + 13.0x + 7.0x^2 + 9.0x^3 + 2.0x^4 + 9.0x^5 + 15.0x^6 + 11.0x^7 + 11.0x^8 + 9.0x^9 + 15.0x^10 + 13.0x^11 + 9.0x^12 + 2.0x^13 + 8.0x^14 + 15.0x^15)
Result(RLWE)  :  R(n=16, q=16, coeffs= 1 + 13x + 7x^2 + 9x^3 + 2x^4 + 9x^5 + 15x^6 + 11x^7 + 11x^8 + 9x^9 + 15x^10 + 13x^11 + 9x^12 + 2x^13 + 8x^14 + 15x^15)
